## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node fe36b4a4-2452-4ab5-8d68-b28bb42f53d5

## Importation of libraries

In [2]:
import keyboard
import time
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from navigation.collision_avoidance import *

## Main script

In [ ]:
map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check the avoidance_modes and the corners. Then, press S.")

while True:
    map.update(True)
    cv.waitKey(20)
    if keyboard.is_pressed('s'):
        if map.found_corners and map.found_destination:
            map.update()
            break
        else:
            print("USER: Must find corners before start")

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

u_old = np.zeros(2)
mu_predict_old = map.robot
cov_predict_old = 100*np.eye(3)
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
avoidance_mode = 0
obj_right = False
obstacle_pos = np.array([0, 0])
destination = map.destination[0:2]
pos_robot = map.robot[0:2]
dt = 0.12

while active == True:
    start_time = time.time()
    print("----------------------")
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    await node.wait_for_variables()

    # Filtering
    # y = map.robot
    # x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, map.found_robot, dt)
    # map.pose_est = np.array([int(x_est[0]), int(x_est[1]), x_est[2]], dtype=object)
    # # map.pose_est, cov_est = map.robot, 100*np.eye(4)
    # print("Estimated pose of Thymio:", map.pose_est)
    # print("Real pose of Thymio:", map.robot)
    # cov_predict_old = cov_est
    # mu_predict_old = map.pose_est
    map.pose_est = map.robot
   
    # Check if final destination moved
    change_dest = ((map.destination[1]-destination[1])**2 + (map.destination[0]-destination[0])**2)*0.5
    if change_dest > 20:
        start = True
        #count = 0
        print("Nouveau chemin destination changé")

    #  Check if the robot has been relocated
    change_robot = ((map.pose_est[1]-pos_robot[1])**2 + (map.pose_est[0]-pos_robot[0])**2)*0.5
    if change_robot > 200:
        start = True
        #count = 0
        print("Nouveau chemin robot changé")
   
    # Global Navigation
    if start == True:
        destination = map.destination[0:2]
        ex_path, corners = compute_visibility_matrix(map.pose_est[0:2], destination, map.obstacles)
        map.avoidance_modes_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        segment_index = 0
        start = False

    #local nav
    u, avoidance_mode, segment_index, obj_right, obstacle_pos = await collision_avoidance(path, node, map.pose_est, avoidance_mode, segment_index, obj_right, obstacle_pos, map.destination[0:2]) # Zacharie
    #print("avoidance_mode", avoidance_mode)

    if not avoidance_mode:
        u, segment_index, end = path_direction(map.pose_est, path, segment_index)
    
    #update robot position
    pos_robot = map.pose_est[0:2]

    await node.set_variables(motors(int(u[0]), int(u[1])))

    u_old = u
    
    end_time = time.time()
    # Update execution time
    dt = end_time - start_time
    print("Execution time:", dt)

    if cv.waitKey(20) & 0xFF==ord('d'):
        map.__del__()
        break

----------------------
distance_to_obstacle 0.0
Execution time: 0.31305408477783203
----------------------
distance_to_obstacle 0.0
Execution time: 0.11540627479553223
----------------------
distance_to_obstacle 1.4142135623730951
Execution time: 0.12064456939697266
----------------------
distance_to_obstacle 6.324555320336759
Execution time: 0.11913728713989258
----------------------
distance_to_obstacle 10.198039027185569
Execution time: 0.12217402458190918
----------------------
distance_to_obstacle 18.24828759089466
Execution time: 0.12396454811096191
----------------------
distance_to_obstacle 24.186773244895647
Execution time: 0.12040066719055176
----------------------
distance_to_obstacle 28.284271247461902
Execution time: 0.12352252006530762
----------------------
distance_to_obstacle 40.01249804748511
Execution time: 0.11835026741027832
----------------------
distance_to_obstacle 45.0
Execution time: 0.12250900268554688
----------------------
distance_to_obstacle 53.0848377599

## Thymio velocity measurements

In [ ]:
from thymio.control import *
import matplotlib.pyplot as plt

active = True
v_l = []
v_r = []

while active:
    await node.wait_for_variables()
    await node.set_variables(motors(100, 100))
    v = get_thymio_velocity(node)
    v_l.append(v[0])
    v_r.append(v[1])
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

v_l_mean = np.mean(v_l[10:])
v_l_var = np.var(v_l[10:])

v_r_mean = np.mean(v_r[10:])
v_r_var = np.var(v_r[10:])

print("Left vel measured:", v_l[10:])
print("Left vel mean:", v_l_mean)
print("Left vel variance:", v_l_var)

nb_iteration = np.linspace(0, len(v_l[10:]), len(v_l[10:]))
plt.plot(nb_iteration, v_l[10:])
plt.title("Left motor velocity covariance in measurement")
plt.show()

print("Right vel measured:", v_r[10:])
print("Right vel mean:", v_r_mean)
print("Right vel variance:", v_r_var)

nb_iteration = np.linspace(0, len(v_r[10:]), len(v_r[10:]))
plt.plot(nb_iteration, v_r[10:])
plt.title("Right motor velocity covariance in measurement")
plt.show()

## Thymio position measurements

In [ ]:
import matplotlib.pyplot as plt

map = Map()

robot_positions_x = []
robot_positions_y = []
robot_positions_theta = []
map.update(True)
while True:
    start = time.time()
    map.update()
    robot_position_x = map.robot[0]
    robot_positions_x.append(robot_position_x)

    robot_position_y = map.robot[1]
    robot_positions_y.append(robot_position_y)
    
    robot_position_theta = map.robot[2] 
    robot_positions_theta.append(robot_position_theta)
    end = time.time()
    dt = end - start
    print(dt)

    if cv.waitKey(20) & 0xFF==ord("d"):
        map.__del__()
        break

robot_positions_x_mean = np.mean(robot_positions_x[10:])
robot_positions_x_var = np.var(robot_positions_x[10:])

robot_positions_y_mean = np.mean(robot_positions_y[10:])
robot_positions_y_var = np.var(robot_positions_y[10:])

robot_positions_theta_mean = np.mean(robot_positions_theta[10:])
robot_positions_theta_var = np.var(robot_positions_theta[10:])

print("Position x measured:", robot_positions_x[10:])
print("Position x mean:", robot_positions_x_mean)
print("Position x variance:", robot_positions_x_var)

nb_iteration = np.linspace(0, len(robot_positions_x[10:]), len(robot_positions_x[10:]))
plt.plot(nb_iteration, robot_positions_x[10:])
plt.title("Position x covariance in measurement")
plt.show()

print("Position y measured:", robot_positions_y[10:])
print("Position y mean:", robot_positions_y_mean)
print("Position y variance:", robot_positions_y_var)

plt.plot(nb_iteration, robot_positions_y[10:])
plt.title("Position y covariance in measurement")
plt.show()

print("Position theta measured:", robot_positions_theta[10:])
print("Position theta mean:", robot_positions_theta_mean)
print("Position theta variance:", robot_positions_theta_var)

plt.plot(nb_iteration, robot_positions_theta[10:])
plt.title("Position theta covariance in measurement")
plt.show()

0.014998912811279297
0.008168935775756836
0.005002260208129883
0.006954193115234375
0.008998394012451172
0.012611150741577148
0.016945838928222656
0.005996227264404297
0.005016326904296875
0.0039980411529541016
0.00500035285949707
0.005742549896240234
0.008004426956176758
0.009998798370361328
0.004999876022338867
0.0050160884857177734
0.004995107650756836
0.0050008296966552734
0.005002498626708984
0.020081281661987305
0.009144783020019531
0.00500178337097168
0.005009651184082031
0.005015373229980469
0.003999471664428711
0.005038738250732422
0.0050694942474365234
0.0050046443939208984
0.0050051212310791016
0.005083560943603516
0.006490230560302734
0.024708271026611328
0.011385679244995117
0.005002498626708984
0.005021572113037109
0.008999109268188477
0.005003213882446289
0.004995107650756836
0.0049991607666015625
0.006593465805053711
0.008000850677490234
0.008466005325317383
0.011067390441894531
0.013177633285522461
0.0050122737884521484
0.003995180130004883
0.005000591278076172
0.00500

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-27410],
 '_imot': [-162, -184],
 '_integrator': [-1079, -1177],
 '_productId': [8],
 '_vbat': [690, 688],
 'acc': [3, 0, 21],
 'acc._tap': [32],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10674, 9159, 13755, 13702, 15106],
 'buttons._noise': [58, 40, 62, 60, 57],
 'buttons._raw': [10658, 9157, 12878, 13710, 15109],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-27410],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [283],
 'mic.intensity': [65],
 'mic.threshold': [0],
 'motor.left.pwm': [-421],
 'motor.left.speed': [89],
 'motor.left.target': [100],
 'motor.right.pwm': [-438],
 'motor.right.speed': [82],
 'motor.right.targ